# Cleaning Eniac data - orderlines

In [1]:
import pandas as pd
from functools import wraps
import datetime as dt

In [2]:
orderlines = pd.read_csv('data/eniac/orderlines.csv')
products = pd.read_csv('data/eniac/products.csv')
orders = pd.read_csv('data/eniac/orders.csv')

In [3]:
temp = orderlines.copy()

In [4]:
# Function that makes logs in the pipeline
def log_step(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        tic = dt.datetime.now()
        result = func(*args, **kwargs)
        time_taken = str(dt.datetime.now() - tic)
        print(f"just ran step {func.__name__} shape={result.shape} took {time_taken}s")
        return result
    return wrapper

In [5]:
# Function that returns copy of DataFrame
@log_step
def start_pipeline(df):
    return df.copy()

## Orderlines info

In [6]:
#Function that returns number of missing values
def check_missing_values(data): 
    print('Missing values:' + '\n' + str(data.isna().sum()))

#Function that returns number of duplicated rows
def check_duplicates(data): 
    print('Duplicated rows: ', data.duplicated().sum())

def check_table(df):
    check_missing_values(df)
    check_duplicates(df)
    df.info()

In [7]:
check_table(orderlines)

Missing values:
id                  0
id_order            0
product_id          0
product_quantity    0
sku                 0
unit_price          0
date                0
dtype: int64
Duplicated rows:  0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB


### Dropping columns

I have decided to drop column product_id because is outdated and real id for products is 'sku' column

In [8]:
# Function that drops columns from a list of columns
@log_step
def drop_columns(df, columns):
    return df.drop(columns,axis=1)

In [9]:
columns_to_drop = ['product_id']

### Changing types 

I have to change types of 'date' (to DateTime) and 'unit_price'  (to float)

#### change 'date' to DateTime

In [10]:
#function that converts DataSeries type to dataFrame
@log_step
def toDateTime(df,column):
    df[column] = pd.to_datetime(df[column])
    return df

In [11]:
col_to_datetime = 'date'

It seems like unit_price had some problems with decimals, decimal spaces...so I have created clean_unit_price function to clean it. After that I was able to change types to float.

#### clean unit_price

In [12]:
#Function for cleaning price and promo_price columns
@log_step
def clean_unit_price(df):
    df['unit_price'] = df['unit_price'].str.rsplit('.',1).apply(lambda x : x[0]+ ',' + x[1]).str.replace('.','',regex=True).str.replace(',','.',regex=True)
    return df

#### change unit_price to float

In [13]:
#Function that converts DataSeries type to float
@log_step
def tofloat(df,column):
    df['unit_price'] = df['unit_price'].astype(float)
    return df

In [14]:
col_to_float = 'unit_price'

### Changing names

I have decided to change name of the column 'id_order' to 'order_id' for easier later merge with orders table

In [15]:
#Function that renames column from old to new
@log_step
def change_col_name(df,old,new):
    return df.rename(columns={old : new})

In [16]:
old='id_order'
new='order_id'

### Dealing with 'ghosts'

Check how many orders from orderlines don't exist in orders by order_id (234- delete them)

In [17]:
# (p.
#     assign(check_orders = p.order_id.isin(orders.order_id)).
#     query("check_orders==False")
# )

In [18]:
@log_step
def delete_orderlines_not_in_orders(df):
    df = (df.
            assign(check_orders = df.order_id.isin(orders.order_id)).
            query("check_orders==True")
            )
    return df.drop('check_orders',axis=1)

### Creating new column 'total_sum' for each orderline (quant*unit_price)

This column could be useful in further exploration so I want it to be permanent column

In [21]:
@log_step
def create_total_sum(df):
    return df.assign(total_sum = df.unit_price * df.product_quantity)

### Dealing with outliers based on big difference between total_paid and total_sum

In [110]:
@log_step
def deleting_outliers(df,df_orders):
    temp = (  df
            .groupby('order_id')
            .agg({'total_sum':'sum'})
            .merge(orders,how='left', left_on='order_id',right_on='order_id')
            
         )
    temp = temp.assign(price_difference = temp.total_paid - temp.total_sum)
    not_outliers = temp[(temp.price_difference > -0.1) & (temp.price_difference < 150)]['order_id'].tolist()
    outliers = temp[(temp.price_difference <= -0.1) | (temp.price_difference >= 150)]['order_id'].tolist()
    df_orders = df_orders[~df_orders.order_id.isin(outliers)]
    df_orders.to_csv('data/eniac/orders_without_outliers.csv',index=False)
    return df[df.order_id.isin(not_outliers)]    

## Pipeline

In [108]:
p = (orderlines
     .pipe(start_pipeline)
     .pipe(drop_columns,columns_to_drop)
     .pipe(toDateTime,col_to_datetime)
     .pipe(clean_unit_price)
     .pipe(tofloat,col_to_float)
     .pipe(change_col_name,old,new)
     .pipe(delete_orderlines_not_in_orders)
     .pipe(create_total_sum)
     .pipe(deleting_outliers,orders)
)

just ran step start_pipeline shape=(293983, 7) took 0:00:00.013967s
just ran step drop_columns shape=(293983, 6) took 0:00:00.015954s
just ran step toDateTime shape=(293983, 6) took 0:00:00.134442s
just ran step clean_unit_price shape=(293983, 6) took 0:00:00.754899s
just ran step tofloat shape=(293983, 6) took 0:00:00.040921s
just ran step change_col_name shape=(293983, 6) took 0:00:00.004992s
just ran step delete_orderlines_not_in_orders shape=(293749, 6) took 0:00:00.035936s
just ran step create_total_sum shape=(293749, 7) took 0:00:00.007015s
just ran step deleting_outliers shape=(292854, 7) took 0:00:00.712701s


In [ ]:
# check_table(p)

## Creating new clean csv

In [111]:
p.to_csv('data/eniac/orderlines_clean.csv',index=False)

### Just some exploring code

Merging orderlines with orders and adding column price_difference

In [104]:
merged = (  p
            .groupby('order_id')
            .agg({'total_sum':'sum'})
            .merge(orders,how='left', left_on='order_id',right_on='order_id')
         )

In [105]:
merged = merged.assign(price_difference = merged.total_paid - merged.total_sum)

In [106]:
merged.price_difference.describe()

count    204211.000000
mean          1.417331
std           3.047138
min          -0.090000
25%           0.000000
50%           0.000000
75%           0.010000
max         132.210000
Name: price_difference, dtype: float64

In [90]:
merged[merged.price_difference > 150].count()

order_id            24
total_sum           24
created_date        24
total_paid          24
state               24
price_difference    24
dtype: int64

In [91]:
merged[merged.price_difference < -0.1].count()

order_id            456
total_sum           456
created_date        456
total_paid          456
state               456
price_difference    456
dtype: int64

Final decision:

In [92]:
outliers = merged[(merged.price_difference <= -0.1) | (merged.price_difference >= 150)]['order_id'].tolist()

In [93]:
len(outliers)

480

In [68]:
p[p.order_id.isin(non_outliers)]

,id,order_id,product_quantity,sku,unit_price,date,total_sum
0,1119109,299539,1,OTT0133,18.99,2017-01-01 00:07:19,18.99
1,1119110,299540,1,LGE0043,399.00,2017-01-01 00:19:45,399.00
2,1119111,299541,1,PAR0071,474.05,2017-01-01 00:20:57,474.05
3,1119112,299542,1,WDT0315,68.39,2017-01-01 00:51:40,68.39
4,1119113,299543,1,JBL0104,23.74,2017-01-01 01:06:38,23.74
...,...,...,...,...,...,...,...
293978,1650199,527398,1,JBL0122,42.99,2018-03-14 13:57:25,42.99
293979,1650200,527399,1,PAC0653,141.58,2018-03-14 13:57:34,141.58
293980,1650201,527400,2,APP0698,9.99,2018-03-14 13:57:41,19.98
293981,1650202,527388,1,BEZ0204,19.99,2018-03-14 13:58:01,19.99
